# Preprocessing and EDA

In [1]:
import nltk
import re
import string
import pandas as pd
import numpy as np
from pprint import pprint
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
from gensim.corpora import MmCorpus
from gensim.utils import simple_preprocess
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel

In [16]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [17]:
df = pd.read_csv("bbc-text.csv")
stopword_list = stopwords.words("english")
stopword_list = stopword_list + ["said", "also", "would", "first", "last", "one"]
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

In [18]:
print(set(df['category'].values))

{'tech', 'politics', 'sport', 'business', 'entertainment'}


In [19]:
# Get most frequent words
def get_stats(words, num_words=200):
    #words = [word for word in words if word not in stopword_list]
    #words = [word for word in words if re.search("[A-Za-z]", word)]
    freq_dist = FreqDist(words)
    return freq_dist

In [20]:
def preprocess(df):
    print(df['text'])
    df['word_list'] = df['text'].apply(lambda x: nltk.tokenize.word_tokenize(x))
    df['word_list'] = df['word_list'].apply(lambda x: [word for word in x if word not in stopword_list])
    df['word_list'] = df['word_list'].apply(lambda x: [word for word in x if re.search("[A-Za-z]", word)])
    df['lemmas'] = df['word_list'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    return df    

In [21]:
df = preprocess(df)
print(df)

0       tv future in the hands of viewers with home th...
1       worldcom boss  left books alone  former worldc...
2       tigers wary of farrell  gamble  leicester say ...
3       yeading face newcastle in fa cup premiership s...
4       ocean s twelve raids box office ocean s twelve...
                              ...                        
2220    cars pull down us retail figures us retail sal...
2221    kilroy unveils immigration policy ex-chatshow ...
2222    rem announce new glasgow concert us band rem h...
2223    how political squabbles snowball it s become c...
2224    souness delight at euro progress boss graeme s...
Name: text, Length: 2225, dtype: object
           category                                               text  \
0              tech  tv future in the hands of viewers with home th...   
1          business  worldcom boss  left books alone  former worldc...   
2             sport  tigers wary of farrell  gamble  leicester say ...   
3             sport  yeadi

In [22]:
flattened = [word for article in df['word_list'].values for word in article]
fd = get_stats(flattened)
#print(fd.most_common(num_words))

In [23]:
# N-grams
def get_ngrams(df, input_col_name, output_col_name, ngram_func):
    df[output_col_name] = df[input_col_name].apply(lambda x: list(ngram_func(x)))
    df[output_col_name] = df[output_col_name].apply(lambda x: [bigram for bigram in x 
                                                   if bigram[0] not in stopword_list
                                                  and bigram[1] not in stopword_list])
    df[output_col_name] = df[output_col_name].apply(lambda x: [bigram for bigram in x 
                                                   if re.search("[A-Za-z]", bigram[0])
                                                  and re.search("[A-Za-z]", bigram[1])])
    df[output_col_name] = df[output_col_name].apply(lambda x: [" ".join(bigram) for bigram in x])
    return df

In [24]:
df = get_ngrams(df, "word_list", "bigrams", nltk.bigrams)
flattened_bigrams = [ngram for article in df['bigrams'].values for ngram in article]
fd = get_stats(flattened_bigrams)
#print(fd.most_common(num_words))

In [25]:
df = get_ngrams(df, "word_list", "trigrams", nltk.trigrams)
flattened_trigrams = [ngram for article in df['trigrams'].values for ngram in article]
fd = get_stats(flattened_trigrams)
#print(fd.most_common(num_words))

In [26]:
#print(df['lemmas'])
flattened_lemmas = [lemma for article in df['lemmas'].values for lemma in article]
fd = get_stats(flattened_lemmas)
#print(fd.most_common(num_words))

# Vectorizing the data

In [27]:
def tokenize_and_lemmatize(sentence):
    tokens = nltk.tokenize.word_tokenize(sentence)
    filtered_tokens = [t for t in tokens if t not in stopword_list and t not in string.punctuation and re.search('[a-zA-Z]', t)]
    lemmas = [lemmatizer.lemmatize(t) for t in filtered_tokens]
    return lemmas

In [28]:
def tokenize_and_stem(sentence):
    tokens = nltk.word_tokenize(sentence)
    filtered_tokens = [t for t in tokens if t not in stopword_list and t not in string.punctuation and re.search('[a-zA-Z]', t)]
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [29]:
def create_vectorizer(text_list):
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.9, min_df=0.02, 
                                       ngram_range=(1,2), stop_words=stopword_list, tokenizer=tokenize_and_stem)
    data = tfidf_vectorizer.fit_transform(text_list)
    return (data, tfidf_vectorizer)

In [30]:
(vect_data, vectorizer) = create_vectorizer(df['text'].values)
#print(list(vectorizer.vocabulary_.keys())[0:100])

/opt/conda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'s", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo'] not in stop_words.
  warnings.warn(


In [31]:
print(list(vectorizer.vocabulary_.keys())[0:100])

['tv', 'futur', 'hand', 'viewer', 'home', 'theatr', 'system', 'digit', 'video', 'record', 'move', 'live', 'room', 'way', 'peopl', 'watch', 'differ', 'five', 'year', 'time', 'accord', 'expert', 'panel', 'gather', 'annual', 'consum', 'electron', 'show', 'discuss', 'new', 'technolog', 'impact', 'favourit', 'us', 'lead', 'trend', 'programm', 'content', 'deliv', 'via', 'network', 'cabl', 'telecom', 'compani', 'broadband', 'servic', 'provid', 'front', 'portabl', 'devic', 'person', 'box', 'like', 'uk', 'allow', 'store', 'play', 'forward', 'want', 'essenti', 'much', 'set', 'big', 'busi', 'japan', 'take', 'europ', 'lack', 'program', 'channel', 'schedul', 'put', 'togeth', 'entertain', 'worri', 'mean', 'term', 'revenu', 'well', 'brand', 'although', 'moment', 'concern', 'rais', 'particular', 'grow', 'happen', 'today', 'see', 'nine', 'month', 'bbc', 'broadcast', 'told', 'news', 'websit', 'issu', 'lost', 'yet', 'press']


# Building the model

In [32]:
def create_and_fit_lda_sklearn(data, num_topics):
    lda = LDA(n_components=num_topics)
    lda.fit(data)
    return lda

In [33]:
lda = create_and_fit_lda_sklearn(vect_data, 5)

In [34]:
def get_most_common_words_for_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names()
    word_dict = {}
    for topic_index, topic in enumerate(model.components_):
        this_topic_words = [words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        word_dict[topic_index] = this_topic_words
    return word_dict

In [35]:
def print_topic_words(word_dict):
    for key in word_dict.keys():
        print(f"Topic {key}")
        print("\t", word_dict[key])

In [37]:
topic_words = get_most_common_words_for_topics(lda, vectorizer, 10)
print_topic_words(topic_words)

Topic 0
	 ['game', 'play', 'win', 'player', 'england', 'club', 'match', 'injuri', 'champion', 'cup']
Topic 1
	 ['trial', 'suspect', 'pcs', 'injur', 'arrest', 'wale', 'b', 'fox', 'format', 'tie']
Topic 2
	 ['mr', 'labour', 'elect', 'parti', 'blair', 'govern', 'tori', 'minist', 'brown', 'lord']
Topic 3
	 ['film', 'award', 'star', 'best', 'music', 'show', 'band', 'actor', 'album', 'nomin']
Topic 4
	 ['compani', 'us', 'firm', 'year', 'use', 'market', 'mr', 'peopl', 'sale', 'share']


# Testing with a new example

In [38]:
new_example = """Manchester United players slumped to the turf 
at full-time in Germany on Tuesday in acknowledgement of what their 
latest pedestrian first-half display had cost them. The 3-2 loss at 
RB Leipzig means United will not be one of the 16 teams in the draw 
for the knockout stages of the Champions League. And this is not the 
only price for failure. The damage will be felt in the accounts, in 
the dealings they have with current and potentially future players 
and in the faith the fans have placed in manager Ole Gunnar Solskjaer. 
With Paul Pogba's agent angling for a move for his client and ex-United 
defender Phil Neville speaking of a "witchhunt" against his former team-mate 
Solskjaer, BBC Sport looks at the ramifications and reaction to a big loss for United."""

In [39]:
def test_new_example(lda, vect, example):
    vectorized = vect.transform([example])
    topic = lda.transform(vectorized)
    print(topic)
    return topic

In [40]:
test_new_example(lda, vectorizer, new_example)

[[0.78915453 0.02937867 0.02984918 0.02977619 0.12184142]]


array([[0.78915453, 0.02937867, 0.02984918, 0.02977619, 0.12184142]])

# Save model

In [41]:
def save_model(lda, lda_path, vect, vect_path):
    pickle.dump(lda, open(lda_path, 'wb'))
    pickle.dump(vect, open(vect_path, 'wb'))